In [ ]:
!git clone --depth 1 https://github.com/ACEsuit/mace.git

# Install MACE
!cd mace && pip install .

In [ ]:
#!pip install numpy==1.23.5
import os
import torch
from ase.calculators.emt import EMT
from mace.calculators import mace_mp
from ase.constraints import FixAtoms
from ase.io import read, write
from ase.mep import NEB
from ase.optimize import BFGS, QuasiNewton
import csv

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set base directory where your structures are stored
base_drive_path = "/content/drive/MyDrive/MLIP-NEB-inputs"  # Modify as needed

device = "cuda" if torch.cuda.is_available() else "cpu"

# Recursively walk through all subdirectories under base_drive_path
for root, dirs, files in os.walk(base_drive_path):
    # Check if current directory contains both "initial" and "final" subdirectories
    if 'initial' in dirs and 'final' in dirs:
        structure_path = root
        structure_name = os.path.basename(structure_path)

        # Define the RESULTS directory path and check if results already exist
        save_dir = os.path.join(structure_path, "MACE")
        csv_path = os.path.join(save_dir, "energies.csv")
        if os.path.exists(csv_path):
            print(f"Skipping {structure_name}: {csv_path} already exists.")
            continue

        # Define paths to initial and final POSCAR files
        initial_path = os.path.join(structure_path, "initial", "POSCAR")
        final_path = os.path.join(structure_path, "final", "POSCAR")
        if not os.path.exists(initial_path) or not os.path.exists(final_path):
            print(f"❌ Skipping {structure_name}: Missing POSCAR in initial or final directory")
            continue

        # Create the RESULTS directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        print(f"✅ Processing {structure_name}")
        try:
            initial = read(initial_path)
            final = read(final_path)
        except Exception as e:
            print(f"Error reading POSCAR for {structure_name}: {e}")
            continue

        # Fix atoms with tag > 1
        mask = [atom.tag > 1 for atom in initial]
        initial.set_constraint(FixAtoms(mask=mask))
        final.set_constraint(FixAtoms(mask=mask))

        # Assign calculator (MACE)
        calc = mace_mp(model="large", dispersion=False, default_dtype="float64", device=device)
        initial.calc = calc
        final.calc = calc

        # Relax initial structure
        try:
            qn = QuasiNewton(initial, trajectory=os.path.join(save_dir, "initial.traj"))
            qn.run(fmax=0.05, steps=1000)
        except RuntimeError as e:
            print(f"LineSearch failed during initial relaxation for {structure_name}: {e}")
            continue

        # Relax final structure
        try:
            qn = QuasiNewton(final, trajectory=os.path.join(save_dir, "final.traj"))
            qn.run(fmax=0.05, steps=1000)
        except RuntimeError as e:
            print(f"LineSearch failed during final relaxation for {structure_name}: {e}")
            continue

        # Save relaxed structures
        write(os.path.join(save_dir, "relaxed_initial.vasp"), initial, format="vasp")
        write(os.path.join(save_dir, "relaxed_final.vasp"), final, format="vasp")

        # Read relaxed structures (to ensure we're using the optimized geometry)
        try:
            initial = read(os.path.join(save_dir, "initial.traj"))
            final = read(os.path.join(save_dir, "final.traj"))
        except Exception as e:
            print(f"Error reading trajectories for {structure_name}: {e}")
            continue

        # Prepare images for NEB calculation
        images = [initial] + [initial.copy() for _ in range(5)] + [final]
        for image in images:
            image.calc = mace_mp(model="large", dispersion=False, default_dtype="float64", device=device)
            image.set_constraint(FixAtoms(mask=mask))

        try:
            neb = NEB(images, parallel=True, k=5, method='eb')
            neb.interpolate('idpp')
        except Exception as e:
            print(f"NEB interpolation failed for {structure_name}: {e}")
            continue

        try:
            qn = BFGS(neb, trajectory=os.path.join(save_dir, "neb.traj"))
            qn.run(fmax=0.05, steps=1000)
        except RuntimeError as e:
            print(f"LineSearch failed during NEB optimization for {structure_name}: {e}")
            continue

        # Store energies and save NEB images
        energies = []
        with open(csv_path, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Image Index", "Energy (eV)"])
            for i, image in enumerate(images):
                try:
                    energy = image.get_potential_energy()
                except Exception as e:
                    print(f"Error computing energy for image {i} of {structure_name}: {e}")
                    energy = None
                energies.append(energy)
                write(os.path.join(save_dir, f"neb_optimized_{i}.vasp"), image, format="vasp")
                writer.writerow([i, energy])

        if None in energies:
            print(f"Skipping migration barrier calculation for {structure_name} due to energy errors")
            continue

        migration_barrier = max(energies) - min(energies)
        print(f"{structure_name} - Migration Barrier: {migration_barrier:.6f} eV")
        with open(csv_path, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([])
            writer.writerow(["Migration Barrier", migration_barrier])

print("✅ All NEB Calculations complete!")





